In [1]:
import keras
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers

from models.transformer import create_vit_model, fit_vit_model
from models.utils.get_train_data import get_average_green_band_data
from models.utils.root_mean_squared_error import root_mean_squared_error
import os.path as osp
import csv
import data
import matplotlib.pyplot as plt

In [2]:
from models.utils.get_n_worst_predictions import get_worst_predictions_from_model

train_data_path = osp.join(osp.dirname(data.__file__), "forest-biomass")
with open(osp.join(osp.dirname(data.__file__), 'patch_names'), newline='') as f:
    reader = csv.reader(f)
    patch_name_data = list(reader)
patch_names = patch_name_data[0]

X_all, y_all, selected_patch_names = get_average_green_band_data(patch_names, train_data_path)

x_train, x_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=42)
x_train = x_train.reshape(x_train.shape[0], 256, 256, 1)
x_test = x_test.reshape(x_test.shape[0], 256, 256, 1)
print("Done!")

print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")


KeyboardInterrupt



In [ ]:
projection_dim = 16
patch_size = 16  # 6  # Size of the patches to be extract from the input images
image_size = 256  # 72  # We'll resize input images to this size
num_patches = (image_size // patch_size) ** 2
model = create_vit_model(input_shape=(256, 256, 1),
                         x_train=x_train,
                         patch_size=patch_size,
                         num_patches=num_patches,
                         projection_dim=projection_dim,
                         transformer_layers=16,
                         num_heads=4,
                         transformer_units=[projection_dim * 2, projection_dim, ],
                         mlp_head_units=[4096, 2048, 1024],
                         learning_rate=0.001,
                         weight_decay=0.0001)

In [ ]:
fitted_model = fit_vit_model(model=model, x_train=x_train, y_train=y_train, batch_size=16,
                             num_epochs=30, validation_split=0.1, save_checkpoint=False)

In [ ]:
worst_n = get_worst_predictions_from_model(fitted_model, 500)
print(worst_n[0])

In [ ]:
pred, label, score, patch_name = worst_n[0]

In [ ]:
score.numpy()

In [ ]:
def plot_pred_and_label(pred, label, score, patch_name):
    plt.subplot(2, 2, 1)
    plt.imshow(pred) #, vmin=0, vmax=500)
    plt.colorbar()

    plt.subplot(2, 2, 2)
    plt.imshow(label) #, vmin=0, vmax=500)
    plt.colorbar()

    plt.subplot(2, 2, 3)
    plt.imshow(pred, vmin=0, vmax=500)
    plt.colorbar()

    plt.subplot(2, 2, 4)
    plt.imshow(label, vmin=0, vmax=500)
    plt.colorbar()

    plt.suptitle("RMSE:" + str(score) + " of patch: " + patch_name)
    plt.show()

In [ ]:
for pred, label, score, patch_name in worst_n:
    plot_pred_and_label(pred, label, score.numpy(), patch_name)

In [ ]:
def visualize_patch(patch_name):
    for month in range(0, 12):
        try:
            patch_data = np.load(osp.join(train_data_path, patch_name, str(month), "S2", "1.npy"), allow_pickle=True)
        except IOError as e:
            continue

        plt.imshow(patch_data)
        plt.colorbar()
        plt.show()

visualize_patch("11feb2e5")

In [2]:
from models.segmenter import train, load_model

# train("Unet", "efficientnet-b7", 10, 0.9, "gpu")
load_model("Unet", "efficientnet-b7", 10, version=None)


Getting train data...


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: tb_logs\Unet_efficientnet-b7
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name  | Type | Params
-------------------------------
0 | model | Unet | 67.1 M
-------------------------------
67.1 M    Trainable params
0         Non-trainable params
67.1 M    Total params
268.407   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "C:\Users\Team Epoch A\Documents\Epoch III\forestbiomass\venv\lib\site-packages\torch\utils\data\_utils\worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "C:\Users\Team Epoch A\Documents\Epoch III\forestbiomass\venv\lib\site-packages\torch\utils\data\_utils\fetch.py", line 61, in fetch
    return self.collate_fn(data)
  File "C:\Users\Team Epoch A\Documents\Epoch III\forestbiomass\venv\lib\site-packages\torch\utils\data\_utils\collate.py", line 265, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
  File "C:\Users\Team Epoch A\Documents\Epoch III\forestbiomass\venv\lib\site-packages\torch\utils\data\_utils\collate.py", line 143, in collate
    return [collate(samples, collate_fn_map=collate_fn_map) for samples in transposed]  # Backwards compatibility.
  File "C:\Users\Team Epoch A\Documents\Epoch III\forestbi